In [1]:
library(tidyverse)
options(bedtools.path = "/share/home/mliu/anaconda3/envs/R-kernel/bin")
library(bedtoolsr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [26]:
R2bed_path = "../../processed/CH001/ct/CH001.ct.R2_5.bed.gz"
max_sep_distance = 10
min_mapping_qual = 30

In [7]:
R2bed <- read_table(R2bed_path,col_names = F) %>% as.data.frame()


── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double(),
  X3 = col_double(),
  X4 = col_character(),
  X5 = col_double(),
  X6 = col_character()
)



In [15]:
R2bed %>% head()

,X1,X2,X3,X4,X5,X6
,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,chr21,47984457,47984489,A00358:815:HKYVYDSX3:4:1101:10167:28776,60,+
2,chr1,61967364,61967450,A00358:815:HKYVYDSX3:4:1101:10167:28776,60,-
3,chr2,76105739,76105849,A00358:815:HKYVYDSX3:4:1101:10176:13072,60,-
4,chr12,115324439,115324497,A00358:815:HKYVYDSX3:4:1101:10411:27884,0,-
5,chr2,9321643,9321755,A00358:815:HKYVYDSX3:4:1101:10447:5869,60,-
6,chr11,92778236,92778342,A00358:815:HKYVYDSX3:4:1101:10547:20165,0,+


In [16]:
R2bed <- R2bed %>% mutate(start = ifelse(X6=="+",X2,X3),end = start+1) %>% select(X1,start,end,X4,X5,X6) %>% arrange(X1,start,end)


In [24]:
R2bed %>% head()

,X1,start,end,X4,X5,X6
,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,chr1,10001,10002,A00358:815:HKYVYDSX3:4:1140:7554:22748,0,+
2,chr1,10002,10003,A00358:815:HKYVYDSX3:4:1261:26512:28776,0,+
3,chr1,10003,10004,A00358:815:HKYVYDSX3:4:1668:10999:14497,0,+
4,chr1,10007,10008,A00358:815:HKYVYDSX3:4:1102:11171:32424,0,+
5,chr1,10008,10009,A00358:815:HKYVYDSX3:4:1277:22752:11271,0,+
6,chr1,10009,10010,A00358:815:HKYVYDSX3:4:2218:14687:8797,0,+


In [20]:
R2bed_cluster <- bedtoolsr::bt.cluster(R2bed,d=max_sep_distance)

In [25]:
names(R2bed_cluster) <- c("chrom","start","end","readid","mapping_qual","strand","cluster")

In [27]:
readid <- R2bed_cluster %>% filter(mapping_qual >= min_mapping_qual) %>% group_by(cluster) %>%
                    arrange(desc(mapping_qual)) %>%slice(1) %>% ungroup() %>% select(readid) %>% unique()

In [ ]:
R1.tabfastq <- read_table2(R1.tabfastq_path,col_names = c("readid","index","seq","qual")) %>% right_join(readid) %>% select(-index)

In [ ]:
R1 = read_table2(R1.tabfastq_path,col_names = c("readid","index","seq","qual"))